In [1]:
tx_start_subset = 1
tx_end_subset = 500000

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd

auth_provider = PlainTextAuthProvider(username='admin', password='admin123')
cluster = Cluster(['35.238.43.195'], port=9042, auth_provider=auth_provider)
session = cluster.connect('bitcoin')

In [3]:
from pyvis import network as net
import networkx as nx

In [4]:
def get_num_rows(node_id):
    query = 'SELECT * from node_info where add_id={}'.format(node_id)
    rows = session.execute(query)
    info = pd.DataFrame(list(rows))
    num_rows = info['num_rows'][0]
    # print(info)
    return num_rows, info

In [5]:
def get_recent_data(node_id):
    num_rows,_ = get_num_rows(node_id)
    block_id = int(num_rows/100)
    for i in range(block_id, -1, -1):
        query = 'SELECT * from transactions where add_id={} and block_id={} and txid>{} and txid<{}'.format(node_id, i, tx_start_subset, tx_end_subset)
        rows = session.execute(query)
        df = pd.DataFrame(list(rows))
        if len(df)<=0:
            has_data = 0
        else:
            df = df.sort_values(by=['timestamp'], ascending=False)
            df['amount'] = round(df['amount']*0.00000001,2)
            # print(df[0:40])
            has_data = 1
            break
    if not has_data:
        print("Data not found!!")
        return None
    return df

In [6]:
def get_role_based_data(df):
    sdf = pd.DataFrame()
    sdf['source'] = df.loc[df['role']=='source']['add_id']
    sdf['target'] = df.loc[df['role']=='source']['nbr_id']
    tdf = pd.DataFrame()
    tdf['source'] = df.loc[df['role']=='target']['nbr_id']
    tdf['target'] = df.loc[df['role']=='target']['add_id']
    #gdf = sdf.append(tdf)
    return sdf, tdf

In [7]:
# size = len(gdf['source'])
# gdf['color'] = ['red' for i in range(size)]
# gb = net.Network(notebook=True, directed=True)
# new = nx.from_pandas_edgelist(gdf, create_using=nx.DiGraph())
# #nx.draw(new, nx.circular_layout(new), edges=new.edges(), edge_color='red')
# gb.from_nx(new)
# gb.show('g.html')

In [19]:
def plot_temporal(df):
    import matplotlib.pyplot as plt
    fig = plt.figure()
    p = fig.add_subplot()
    p.plot_date(df.loc[df['role']=='source']['timestamp'], df.loc[df['role']=='source']['amount'], color='red', label='outgoing')
    p.plot_date(df.loc[df['role']=='target']['timestamp'], df.loc[df['role']=='target']['amount'], color='green', label='incoming')
    p.legend()
    plt.xticks(rotation=45)
    plt.show()

In [14]:
def get_khop_data(n_hop, sdf, tdf):
    lim = min(len(sdf['target']),5)
    for k in range(n_hop):
        target_nodes = sdf['target'][len(sdf['target'])-lim:]
        lim = 0
        for node in target_nodes:
            rdf = get_recent_data(node)
            if rdf is None:
                continue
            kdf = pd.DataFrame()
            kdf['source'] = rdf.loc[rdf['role']=='source']['add_id']
            kdf['target'] = rdf.loc[rdf['role']=='source']['nbr_id']
            klim = min(len(kdf['source']),5)
            lim = lim + klim
            sdf = sdf.append(kdf[:][len(kdf['source'])-lim:])
    print(sdf.shape)
    lim = min(len(tdf['source']),5)
    for k in range(n_hop):
        source_nodes = tdf['source'][len(tdf['source'])-lim:]
        lim = 0
        for node in target_nodes:
            rdf = get_recent_data(node)
            if rdf is None:
                continue
            kdf = pd.DataFrame()
            kdf['source'] = rdf.loc[rdf['role']=='target']['nbr_id']
            kdf['target'] = rdf.loc[rdf['role']=='target']['add_id']
            klim = min(len(kdf['target']),5)
            lim = lim + klim
            tdf = tdf.append(kdf[:][len(kdf['target'])-lim:])
    print(tdf.shape)
    gdf = sdf.append(tdf)
    return gdf

In [10]:
def plot_khop(gdf):
    kgb = net.Network(notebook=True, directed=True)
    kplot = nx.from_pandas_edgelist(gdf, create_using=nx.DiGraph())
    #edges=nx.draw_networkx_edges(kplot,pos=nx.spring_layout(kplot))
    kgb.from_nx(kplot)
    return kgb

In [21]:
node_id = 265689
k=3
_, info = get_num_rows(node_id)
print(info)
df = get_recent_data(node_id)
print(df[0:40])
plot_temporal(df)
sdf, tdf = get_role_based_data(df)
gdf = get_khop_data(k, sdf, tdf)
print(gdf.shape)
kgb = plot_khop(gdf)
kgb.show('graph_{}_{}.html'.format(node_id, k))

OperationTimedOut: errors={'35.238.43.195:9042': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=35.238.43.195:9042

In [ ]:
node_1 = 744
node_2 = 9
query = "select * from vector_embeddings where add_id in ({},{})".format(node_1, node_2)
rows = session.execute(query)
edf = pd.DataFrame(list(rows))
print(edf['add_id'][0])
print(edf['vector'][0])
print(edf['add_id'][1])
print(edf['vector'][1])